In [66]:
import numpy as np
import os
import re

from collections import defaultdict

In [7]:
# set experiment
exp='/exp_1a'

In [67]:
def sent2idx(sents:list, w2i:dict):
    all_sents = []
    for sent in sents:
        indices = np.zeros(len(sent), dtype=int)
        for i, w in enumerate(sent):
            indices[i] += w2i[w]
        all_sents.append(indices)
    return np.array(all_sents)

In [68]:
def sort_dict(some_dict:dict): return dict(sorted(some_dict.items(), key=lambda kv:kv[1], reverse=True))

In [73]:
def load_dataset(exp:str, split:str, subdir:str='./data'):
    file = subdir+exp+split+'/'+os.listdir(subdir+exp+split).pop()
    cmd_start = 'IN:'
    act_start = 'OUT:'
    cmds, acts = [], []
    cmd_vocab, act_vocab = defaultdict(int), defaultdict(int)
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            cmd = line[line.index(cmd_start)+len(cmd_start):line.index(act_start)].strip().split()
            act = line[line.index(act_start)+len(act_start):].strip().split()
            for w in cmd: cmd_vocab[w] += 1
            for w in act: act_vocab[w] += 1
            cmds.append(cmd)
            acts.append(act)
    cmd_vocab = sort_dict(cmd_vocab)
    act_vocab = sort_dict(act_vocab)
    # create w2i mappings
    w2i_cmds = {w: i + 3 for i, w in enumerate(cmd_vocab.keys())}
    w2i_cmds.update({'UNK': 0})
    w2i_cmds.update({'<SOS>': 1})
    w2i_cmds.update({'<EOS>': 2})
    # add number of unique words in cmd vocab to action indices (to not end up with a shared index vocabulary)
    w2i_acts = {w: i + len(w2i_cmds) for i, w in enumerate(act_vocab.keys())}
    return cmd_vocab, w2i_cmds, cmds, act_vocab, w2i_acts, acts

In [70]:
cmd_vocab, w2i_cmds, cmds, act_vocab, w2i_acts, acts = load_dataset(exp='/exp_1a', split='/train')

In [71]:
train_commands = sent2idx(cmds, w2i_cmds)
train_actions = sent2idx(acts, w2i_acts)

In [72]:
train_commands

array([array([ 9,  5,  1,  3,  8, 13,  5,  1,  4]),
       array([11,  5,  2,  7, 12,  1]), array([12,  7, 11,  6,  1,  3]),
       ..., array([11,  1,  4,  8, 13,  6,  1,  3]),
       array([11,  5,  1,  4,  8, 12,  3]),
       array([10,  5,  1,  3,  7,  9,  6,  1,  3])], dtype=object)